In [15]:
import pandas as pd
import numpy as np
import os

INTERIM_DIR = '../data/interim/'
PROCESSED_DIR = '../data/processed/'
ENRICHED_FILE = os.path.join(INTERIM_DIR, 'sales_data_enriched.csv')
CLEAN_FILE = os.path.join(PROCESSED_DIR, 'final_cleaned_data.csv')

In [16]:
dtype_spec = {
    'sale_price': 'object',
    'class': 'str',
    'pin': 'str',
    'pin10': 'str'
}
df = pd.read_csv(ENRICHED_FILE, dtype=dtype_spec)
df.shape

/var/folders/fk/9chv4jr11ln50p4dhsxntpcw0000gn/T/ipykernel_55920/3334969353.py:7: DtypeWarning: Columns (8,15,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ENRICHED_FILE, dtype=dtype_spec)


(877037, 26)

In [17]:
# Convert sale_price to numeric (handles '$' and commas)
df['sale_price'] = df['sale_price'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False)
df['sale_price'] = pd.to_numeric(df['sale_price'], errors='coerce')

In [18]:
# Drop rows where critical enriched features are missing (should be few after initial enrichment)
df.dropna(subset=['sale_price', 'lon', 'lat', 'min_distance_meters'], inplace=True)
print(f"Shape after critical null drops: {df.shape}")

Shape after critical null drops: (877037, 26)


In [19]:
print(df['class'].unique())
RESIDENTIAL_CLASSES = ['202', '203', '204', '205', '206', '207', '208', '209']
df['class'] = df['class'].astype(str).str[:3]
df_filtered = df[df['class'].isin(RESIDENTIAL_CLASSES)].copy()
df_filtered.shape

['234' '203' '211' '241' '202' '295' '299' '210' '212' '205' '100' 'EX'
 '315' '206' '278' '204' '207' '290' '590' '593' '517' '597' '523' '201'
 '522' '529' '297' '599' '318' '208' '209' '592' '213' '391' '314' '591'
 '239' '218' '580' '583' '313' '528' '190' '717' '790' '817' '531' '200'
 '530' '390' '501' '914' '301' '497' '663' '527' '893' '996' '990' '219'
 '915' '224' '321' '663B' '670' '823' '890' '897' '533' '892' '918' '397'
 '532' '822' '589' '829' 'RR' '221' '535' '319' '300' '490' '889' '893A'
 '797' '288' '587' '991' '663A' '396' '880' '899' '417' '491' '893B' '499'
 '729A' '225' '516' '492' '883' '399' '220' '320' '550' 'OA3' '236' '435'
 '693' '294' '765' '791' '526' '670B' '679' '420' '680' '677' '997' '921'
 '500' '913' '428' '833' '638' '792']


(138783, 26)

In [20]:
df_filtered = df_filtered[
    (df_filtered['is_multisale'] == False) &
    (df_filtered['sale_filter_less_than_10k'] == False) &
    (df_filtered['sale_filter_deed_type'] == False)
]
df_filtered.shape

(116972, 26)

In [21]:
df_filtered['sale_date'] = pd.to_datetime(df_filtered['sale_date'], errors='coerce')
RECENT_YEAR = 2018
df_filtered = df_filtered[df_filtered['sale_date'].dt.year >= RECENT_YEAR]
df_filtered.shape

(41468, 26)

In [22]:
lower_bound = df_filtered['sale_price'].quantile(0.01)
upper_bound = df_filtered['sale_price'].quantile(0.99)
df_final = df_filtered[
    (df_filtered['sale_price'] >= lower_bound) & 
    (df_filtered['sale_price'] <= upper_bound)
].copy()
df_final.describe()

,year,township_code,neighborhood_code,sale_date,sale_price,num_parcels_sale,row_id,lon,lat,min_distance_meters
count,40646.000000,40646.000000,40646.000000,40646,4.064600e+04,40646.0,4.064600e+04,40646.000000,40646.000000,40646.000000
mean,2023.034591,47.632043,47764.508390,2023-07-05 08:54:25.738326016,3.133179e+05,1.0,2.064379e+07,-87.725513,41.782261,7474.989752
min,2018.000000,10.000000,10012.000000,2018-01-01 00:00:00,2.630000e+04,1.0,7.087465e+06,-88.027405,41.469928,46.065505
25%,2022.000000,27.000000,27030.000000,2022-05-31 00:00:00,1.600000e+05,1.0,7.590865e+06,-87.792405,41.683882,2022.776948
50%,2024.000000,39.000000,39080.000000,2024-05-13 00:00:00,2.650000e+05,1.0,7.661326e+06,-87.723170,41.776316,4567.916309
75%,2024.000000,72.000000,72030.000000,2024-12-06 00:00:00,3.800000e+05,1.0,7.742782e+06,-87.660475,41.908445,10378.998639
max,2025.000000,77.000000,77170.000000,2025-09-29 00:00:00,1.700000e+06,1.0,9.848472e+07,-87.524919,42.065298,33760.975882
std,2.122462,23.937873,23984.904038,NaN,2.373906e+05,0.0,3.162758e+07,0.092295,0.145744,7456.189297


In [23]:
df_final.to_csv(CLEAN_FILE, index=False)